In [1]:
import pandas as pd
import numpy as np
import torch
from torch import cuda
from torch.utils.data import Dataset, DataLoader
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from load_data import initialize_data
from reading_datasets import read_task
from labels_to_ids import labels_to_ids_tam
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
def train(epoch, train_loader, model, optimizer, device, grad_step = 1, max_grad_norm = 10):
    tr_loss, tr_accuracy = 0, 0
    tr_precision, tr_recall = 0, 0
    tr_f1score = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    optimizer.zero_grad()
    
    for idx, batch in enumerate(train_loader):
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        if (idx + 1) % 20 == 0:
            print('FINSIHED BATCH:', idx, 'of', len(train_loader))

        #loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        output = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += output[0]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
        
        # Compute Precision
        tmp_tr_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0 )
        tr_precision += tmp_tr_precision
        
        # Compute Recall
        tmp_tr_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
        tr_recall += tmp_tr_recall
        
        # Compute f1score
        tmp_tr_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average= 'macro', zero_division=0)
        tr_f1score += tmp_tr_f1score
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=max_grad_norm
        )
        
        # backward pass
        output['loss'].backward()
        if (idx + 1) % grad_step == 0:
            optimizer.step()
            optimizer.zero_grad()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    tr_precision = tr_precision / nb_tr_steps
    tr_recall = tr_recall / nb_tr_steps
    tr_f1score= tr_f1score / nb_tr_steps
    #print(f"Training loss epoch: {epoch_loss}")
    #print(f"Training accuracy epoch: {tr_accuracy}")

    return model

In [3]:
def testing(model, testing_loader, labels_to_ids, device):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    eval_precision, eval_recall = 0, 0
    eval_f1score = 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
     
    
    ids_to_labels = dict((v,k) for k,v in labels_to_ids.items())

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            #loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            output = model(input_ids=ids, attention_mask=mask, labels=labels)

            eval_loss += output['loss'].item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = output[1].view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            
            # Compute Precision
            tmp_eval_precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_precision += tmp_eval_precision
            
            # Compute Recall
            tmp_eval_recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average = 'macro', zero_division=0)
            eval_recall += tmp_eval_recall
            
            # Compute f1score
            tmp_eval_f1score = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='macro', zero_division=0)
            eval_f1score += tmp_eval_f1score

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    eval_precision = eval_precision / nb_eval_steps
    eval_recall = eval_recall / nb_eval_steps
    eval_f1score = eval_f1score / nb_eval_steps
    #print(f"Validation Loss: {eval_loss}")
    #print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions, eval_accuracy, eval_precision, eval_recall, eval_f1score

In [4]:
def main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location):
    #Initialization training parameters
    max_len = 256
    batch_size = 32
    grad_step = 1
    learning_rate = 1e-05
    initialization_input = (max_len, batch_size)

    #Reading datasets and initializing data loaders
    dataset_location = '../datasets/task_a/'

    train_data = read_task(dataset_location , split = 'tam_sentiment_train')
    dev_data = read_task(dataset_location , split = 'tam_sentiment_dev')
    #test_data = read_task(dataset_location , split = 'dev')#load test set
    labels_to_ids = labels_to_ids_tam
    #input_data = (train_data, dev_data, labels_to_ids)

    #Define tokenizer, model and optimizer
    device = 'cuda' if cuda.is_available() else 'cpu' #save the processing time
    if model_load_flag:
        tokenizer = AutoTokenizer.from_pretrained(model_load_location)
        model = AutoModelForSequenceClassification.from_pretrained(model_load_location)
    else: 
        tokenizer =  AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    model.to(device)

    #Get dataloaders
    train_loader = initialize_data(tokenizer, initialization_input, train_data, labels_to_ids, shuffle = True)
    dev_loader = initialize_data(tokenizer, initialization_input, dev_data, labels_to_ids, shuffle = True)
    #test_loader = initialize_data(tokenizer, initialization_input, test_data, labels_to_ids, shuffle = True)#create test loader

    best_dev_acc = 0
    best_test_acc = 0
    best_dev_precision = 0
    best_test_precision = 0
    best_dev_recall = 0
    best_test_recall = 0
    best_dev_f1score = 0
    best_test_f1score = 0
    best_epoch = -1
    
    list_dev_acc = [] 
    list_test_acc = []  
    list_dev_precision = []  
    list_test_precision  = []  
    list_dev_recall = []  
    list_test_recall = []  
    list_dev_f1score = []  
    list_test_f1score = []
    
    for epoch in range(n_epochs):
        start = time.time()
        print(f"Training epoch: {epoch + 1}")

        #train model
        if not model_load_flag:
            model = train(epoch, train_loader, model, optimizer, device, grad_step)
        
        #testing and logging
        labels_dev, predictions_dev, dev_accuracy, dev_precision, dev_recall, dev_f1score = testing(model, dev_loader, labels_to_ids, device)
        print('DEV ACC:', dev_accuracy)
        print('DEV Precision:' , dev_precision)
        print('DEV Recall:' , dev_recall)
        print('DEV F1Score:' , dev_f1score)
        
        list_dev_acc.append(dev_accuracy)     
        list_dev_precision.append(dev_precision)   
        list_dev_recall.append(dev_recall)  
        list_dev_f1score.append(dev_f1score)  
        
        
        #labels_test, predictions_test, test_accuracy, test_precision, test_recall, test_f1score = testing(model, test_loader, labels_to_ids, device)
        #print('TEST ACC:', test_accuracy)
        #print('TEST Precision:' , test_precision)
        #print('TEST Recall:' , test_recall)
        #print('TEST F1Score:' , test_f1score)
        
        #list_test_acc.append(test_accuracy) 
        #list_test_precision.append(test_precision)  
        #list_test_recall.append(test_recall)
        #list_test_f1score.append(test_f1score) 

        #saving model
        if dev_accuracy > best_dev_acc:
            best_dev_acc = dev_accuracy
            #best_test_acc = test_accuracy
        if dev_precision > best_dev_precision:
            best_dev_precision = dev_precision
            #best_test_precision = test_precision
        if dev_recall > best_dev_recall:
            best_dev_recall = dev_recall
            #best_test_recall = test_recall
        if dev_f1score > best_dev_f1score:
            best_dev_f1score = dev_f1score
            #best_test_f1score = test_f1score
            best_epoch = epoch
            
            if model_save_flag:
                os.makedirs(model_save_location, exist_ok=True)
                tokenizer.save_pretrained(model_save_location)
                model.save_pretrained(model_save_location)

        now = time.time()
        print('BEST ACCURACY --> ', 'DEV:', round(best_dev_acc, 5))
        print('BEST PRECISION --> ', 'DEV:', round(best_dev_precision, 5))
        print('BEST RECALL --> ', 'DEV:', round(best_dev_recall, 5))
        print('BEST F1SCORE --> ', 'DEV:', round(best_dev_f1score, 5))
        print('TIME PER EPOCH:', (now-start)/60 )
        print()

    return best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score

In [5]:
if __name__ == '__main__':
    n_epochs = 10
    models = ['bert-base-uncased']
    
    #model saving parameters
    model_save_flag = True
    model_load_flag = False
    
    overall_list_dev_acc = [] 
    overall_list_test_acc = []    
    overall_list_dev_precision = []  
    overall_list_test_precision  = []  
    overall_list_dev_recall = []  
    overall_list_test_recall = []  
    overall_list_dev_f1score = []  
    overall_list_test_f1score = [] 
    
    for i in range(5):
        
        for model_name in models:

            model_save_location = 'saved_models/' + model_name + 'Tamil' + str(i)
            model_load_location = None

            best_dev_acc, best_test_acc, best_epoch, best_dev_precision, best_test_precision, best_dev_recall, best_test_recall, best_dev_f1score, best_test_f1score, list_dev_acc, list_test_acc, list_dev_precision, list_test_precision, list_dev_recall, list_test_recall, list_dev_f1score, list_test_f1score = main(n_epochs, model_name, model_save_flag, model_save_location, model_load_flag, model_load_location)
            
            overall_list_dev_acc.append(list_dev_acc) 
            overall_list_test_acc.append(list_test_acc) 
            overall_list_dev_precision.append(list_dev_precision)  
            overall_list_test_precision.append(list_test_precision) 
            overall_list_dev_recall.append(list_dev_recall)  
            overall_list_test_recall.append(list_test_recall)  
            overall_list_dev_f1score.append(list_dev_f1score)  
            overall_list_test_f1score.append(list_test_f1score) 

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHE

FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss

FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.7247428894042969
Validation loss per 100 evaluation steps: 1.4831513420189961
DEV ACC: 0.5915400124069479
DEV Precision: 0.45302305009214167
DEV Recall: 0.4504718533914533
DEV F1Score: 0.43048206971072484
BEST ACCURACY -->  DEV: 0.64078
BEST PRECISION -->  DEV: 0.48907
BEST RECALL -->  DEV: 0.46894
BEST F1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.1796238422393799
Validation loss per 100 evaluation steps: 1.3846563602438067
DEV ACC: 0.6115074441687345
DEV Precision: 0.45566648860211084
DEV Recall: 0.44739913041446533
DEV F1Score: 0.4252673658000979
BEST ACCURACY -->  DEV: 0.64853
BEST PRECISION -->  DEV: 0.49669
BEST RECA

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Training epoch: 1
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATCH: 339 of 1115
FINSIHED BATCH: 359 of 1115
FINSIHED BATCH: 379 of 1115
FINSIHED BATCH: 399 of 1115
FINSIHED BATCH: 419 of 1115
FINSIHED BATCH: 439 of 1115
FINSIHED BATCH: 459 of 1115
FINSIHED BATCH: 479 of 1115
FINSIHED BATCH: 499 of 1115
FINSIHED BATCH: 519 of 1115
FINSIHED BATCH: 539 of 1115
FINSIHED BATCH: 559 of 1115
FINSIHED BATCH: 579 of 1115
FINSIHED BATCH: 599 of 1115
FINSIHED BATCH: 619 of 1115
FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHE

FINSIHED BATCH: 639 of 1115
FINSIHED BATCH: 659 of 1115
FINSIHED BATCH: 679 of 1115
FINSIHED BATCH: 699 of 1115
FINSIHED BATCH: 719 of 1115
FINSIHED BATCH: 739 of 1115
FINSIHED BATCH: 759 of 1115
FINSIHED BATCH: 779 of 1115
FINSIHED BATCH: 799 of 1115
FINSIHED BATCH: 819 of 1115
FINSIHED BATCH: 839 of 1115
FINSIHED BATCH: 859 of 1115
FINSIHED BATCH: 879 of 1115
FINSIHED BATCH: 899 of 1115
FINSIHED BATCH: 919 of 1115
FINSIHED BATCH: 939 of 1115
FINSIHED BATCH: 959 of 1115
FINSIHED BATCH: 979 of 1115
FINSIHED BATCH: 999 of 1115
FINSIHED BATCH: 1019 of 1115
FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 0.8572461605072021
Validation loss per 100 evaluation steps: 1.0518587632934646
DEV ACC: 0.62044432382134
DEV Precision: 0.48227150990797474
DEV Recall: 0.47194820633491114
DEV F1Score: 0.4494284440199218
BEST ACCURACY -->  DEV: 0.64189
BEST PRECISION -->  DEV: 0.50275
BEST RECALL

FINSIHED BATCH: 1039 of 1115
FINSIHED BATCH: 1059 of 1115
FINSIHED BATCH: 1079 of 1115
FINSIHED BATCH: 1099 of 1115
Validation loss per 100 evaluation steps: 1.521236777305603
Validation loss per 100 evaluation steps: 1.3287577853344454
DEV ACC: 0.6113329714640199
DEV Precision: 0.4820465354385207
DEV Recall: 0.46936587648430717
DEV F1Score: 0.4526651929634106
BEST ACCURACY -->  DEV: 0.64189
BEST PRECISION -->  DEV: 0.50275
BEST RECALL -->  DEV: 0.47195
BEST F1SCORE -->  DEV: 0.45267
TIME PER EPOCH: 25.390082236131033

Training epoch: 10
FINSIHED BATCH: 19 of 1115
FINSIHED BATCH: 39 of 1115
FINSIHED BATCH: 59 of 1115
FINSIHED BATCH: 79 of 1115
FINSIHED BATCH: 99 of 1115
FINSIHED BATCH: 119 of 1115
FINSIHED BATCH: 139 of 1115
FINSIHED BATCH: 159 of 1115
FINSIHED BATCH: 179 of 1115
FINSIHED BATCH: 199 of 1115
FINSIHED BATCH: 219 of 1115
FINSIHED BATCH: 239 of 1115
FINSIHED BATCH: 259 of 1115
FINSIHED BATCH: 279 of 1115
FINSIHED BATCH: 299 of 1115
FINSIHED BATCH: 319 of 1115
FINSIHED BATC

In [6]:
print(best_dev_acc)

0.6418850806451613


In [7]:
print(best_epoch)

8


In [8]:
print(best_dev_precision)

0.5027548394596978


In [9]:
print(best_dev_recall)

0.47194820633491114


In [10]:
print(best_dev_f1score)

0.4526651929634106


In [11]:
print(overall_list_dev_acc)

[[0.6266090260545906, 0.6355071339950372, 0.627248759305211, 0.6407800868486353, 0.6247867555831266, 0.6220145781637717, 0.6156172456575683, 0.6142796215880894, 0.5915400124069479, 0.6112748138957816], [0.6291098014888338, 0.6376395781637717, 0.644075682382134, 0.6429319168734492, 0.619823976426799, 0.6270549007444168, 0.6296913771712159, 0.6215105459057072, 0.6039275744416873, 0.5847355769230769], [0.6292067307692307, 0.6374457196029776, 0.6441919975186104, 0.6424278846153846, 0.635061259305211, 0.6332971464019852, 0.6283925248138958, 0.6006707506203475, 0.605924317617866, 0.5971425248138958], [0.6130195409429281, 0.6455490074441687, 0.648534429280397, 0.6437073511166252, 0.6431257754342432, 0.6208320409429281, 0.6201341501240695, 0.6246704404466501, 0.6115074441687345, 0.6165477667493797], [0.6224604528535981, 0.635642834987593, 0.6418850806451613, 0.636883529776675, 0.62044432382134, 0.6312616315136477, 0.6037724875930521, 0.6122634925558313, 0.6113329714640199, 0.6181761786600497]]

In [12]:
print(overall_list_dev_precision)

[[0.4648673010022494, 0.47796759349809714, 0.45545296110696, 0.4890673651077375, 0.48503905431767136, 0.488707984336314, 0.4547713319405384, 0.4705701795631125, 0.45302305009214167, 0.4667558066383406], [0.4593950842888209, 0.48076670870233107, 0.4768655237647192, 0.4755683925155304, 0.48297654118289757, 0.49379085299562936, 0.4790430558291469, 0.46660533020652434, 0.4795315523913803, 0.45229621161762174], [0.4425785121900333, 0.48028744191376305, 0.48700005418483544, 0.4899703065790713, 0.48567820547548957, 0.5005490398692269, 0.48453841665416336, 0.4767771113613413, 0.4728527745863388, 0.45492189314972864], [0.4328232152511329, 0.4810820393512161, 0.48423999654223204, 0.49669245916822313, 0.4842395528016884, 0.469481938948228, 0.47970320323735116, 0.46848011130307415, 0.45566648860211084, 0.445498981781014], [0.4582905176255797, 0.47443402924479644, 0.4952092021316226, 0.5027548394596978, 0.48227150990797474, 0.46687190934498213, 0.46081035799417724, 0.4760842279041562, 0.48204653543

In [13]:
print(overall_list_dev_recall)

[[0.42027309246121386, 0.4460380379080783, 0.46625162722831187, 0.44705355128847646, 0.448249964299865, 0.4689417872301138, 0.44738230020312214, 0.44575791050661684, 0.4504718533914533, 0.4393180322985935], [0.4066761158431517, 0.43108247219493206, 0.4460592545916353, 0.45322338228530334, 0.45683243294220893, 0.45487200443541975, 0.4365372614461878, 0.457146379256243, 0.4524699742999101, 0.4486631486353272], [0.41197460463251595, 0.4535494704493978, 0.46385795868154805, 0.46786235815921423, 0.4843349541841572, 0.4629911374911401, 0.45530158876130167, 0.47548920236344094, 0.4565476072379006, 0.44371348789095655], [0.3975469474250125, 0.44110487005106297, 0.43364125111683927, 0.44654128626099504, 0.4424897678529917, 0.43846816548481876, 0.46679148985407565, 0.4247610505900298, 0.44739913041446533, 0.44622530878635236], [0.4193622972726417, 0.40132073537454593, 0.43967558019963554, 0.45255279784527697, 0.47194820633491114, 0.43337159815692244, 0.4583779124452649, 0.45319637905343996, 0.46

In [14]:
print(overall_list_dev_f1score)

[[0.4065458481446836, 0.4324380835538, 0.43437599492600243, 0.437196012518436, 0.4319129915320516, 0.44984228405977394, 0.427391929559698, 0.4323017756558786, 0.43048206971072484, 0.42774099124184645], [0.39345476049929085, 0.4185346029673007, 0.4331280667430327, 0.43732951862549835, 0.4465680251038883, 0.44378537138923396, 0.42954027832324637, 0.4364404353975123, 0.44226213111125645, 0.42780166953627075], [0.3959477365651797, 0.43485824317140775, 0.4465454374711284, 0.4488416519069074, 0.46075349391287995, 0.45106210062251295, 0.44136601966454964, 0.4491799878706377, 0.44068091175296675, 0.4279912238509796], [0.3784204884205143, 0.42844936764279606, 0.4237273113677032, 0.4419099138908733, 0.4297911024034114, 0.4282386631621183, 0.4475507302750363, 0.4183952439793843, 0.4252673658000979, 0.42502950529771677], [0.40159072353894, 0.39303095458263826, 0.42868939950635576, 0.4473606411313397, 0.4494284440199218, 0.4245924314822645, 0.43421104610943617, 0.4385847393654634, 0.452665192963410

In [15]:
#The best model is 3